<a href="https://colab.research.google.com/github/apchavezr/19.-Procesamiento-del-Lenguaje-Natural/blob/main/Mineria_de_textos_en_noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo 3. minería de textos en noticias

En el ámbito periodístico y empresarial, la minería de textos permite transformar noticias en información estructurada que facilita el análisis de tendencias y la toma de decisiones.

**Texto original:**
“Ecopetrol anunció en Bogotá un acuerdo con Repsol para explorar yacimientos en Meta.”

. Muestra:
* NER con spaCy en español, y
* una extracción de relaciones basada en reglas simples usando dependencias sintácticas.

In [8]:
#import spacy
#from spacy.symbols import nsubj, VERB

nlp = spacy.load("es_core_news_md")

texto = "Ecopetrol anunció en Bogotá un acuerdo con Repsol para explorar yacimientos en Meta."

doc = nlp(texto)

# ---- NER: entidades detectadas
entidades = [(ent.text, ent.label_) for ent in doc.ents]
print("ENTIDADES (texto, etiqueta):")
for e in entidades:
    print(" -", e)

# Mapa simple de etiquetas spaCy → categorías que usamos en el ejemplo
mapa = {"ORG": "Organización", "LOC": "Ubicación", "PER": "Persona", "MISC": "Mención"}
print("\nENTIDADES (normalizadas):")
for ent in doc.ents:
    print(f" - {ent.text} → {mapa.get(ent.label_, ent.label_)}")

# ---- Reglas mínimas para extraer relaciones del ejemplo
# Idea:
#  - Sujeto organizacional de un verbo (nsubj) → posible actor (Ecopetrol)
#  - Sustantivo "acuerdo" → núcleo del evento
#  - prep 'con' desde "acuerdo" → contraparte (Repsol)
#  - prep 'en' desde el verbo principal o desde "acuerdo" → ubicación (Bogotá)
#  - prep 'para' desde "acuerdo" → objetivo (verbo + objeto), y si hay 'en' anidado → ubicación de la acción (Meta)

def buscar_token(doc, lemma=None, lower_text=None, pos=None):
    for t in doc:
        if lemma and t.lemma_.lower() != lemma.lower():
            continue
        if lower_text and t.text.lower() != lower_text.lower():
            continue
        if pos and t.pos_ != pos:
            continue
        return t
    return None

# verbo principal (heurístico: primer VERB con sujeto)
verbo = None
for t in doc:
    if t.pos_ == "VERB" and any(c.dep == nsubj for c in t.children):
        verbo = t
        break

# sujeto organizacional
sujeto_org = None
if verbo:
    for c in verbo.children:
        if c.dep == nsubj:
            # comprobar si intersecta con una entidad ORG
            for ent in doc.ents:
                if ent.label_ == "ORG" and c.i >= ent.start and c.i < ent.end:
                    sujeto_org = ent.text
                    break

# "acuerdo"
acuerdo = buscar_token(doc, lemma="acuerdo", pos="NOUN")

# contraparte con 'con'
contraparte = None
ubicacion_acuerdo = None
objetivo = None
ubicacion_objetivo = None

if acuerdo:
    for child in acuerdo.children:
        if child.dep_ == "case" and child.text.lower() == "con":
            # head del 'con' es el complemento (spaCy español: el 'con' suele ser hijo del complemento con dep 'case')
            obj_con = child.head
            # verificar si es parte de una entidad ORG
            for ent in doc.ents:
                if ent.label_ == "ORG" and obj_con.i >= ent.start and obj_con.i < ent.end:
                    contraparte = ent.text
                    break
        if child.dep_ == "mark" and child.text.lower() == "para":
            # objetivo: buscar hijo verbal "explorar" y su objeto
            for cc in acuerdo.children:
                if cc.pos_ == "VERB":
                    # construir objetivo básico: verbo + objeto
                    objeto = None
                    for cc2 in cc.children:
                        if cc2.dep_ in ("obj", "dobj", "obl") or cc2.dep_ == "nmod":
                            # tomar la frase completa del objeto
                            span = doc[cc2.left_edge.i:cc2.right_edge.i+1]
                            objeto = span.text
                        # hallar 'en' (ubicación del objetivo), si existe
                        if cc2.dep_ == "case" and cc2.text.lower() == "en":
                            ubj = cc2.head
                            ubicacion_objetivo = ubj.text
                    objetivo = (cc.lemma_, objeto)

# ubicación a partir del verbo principal ('en Bogotá')
ubicacion_evento = None
if verbo:
    for tok in verbo.subtree:
        # si hay una entidad LOC dentro del subárbol con 'en' previo
        if tok.text.lower() == "en":
            loc = tok.head
            # si el head es un nombre propio/locación:
            for ent in doc.ents:
                if ent.label_ in ("LOC", "GPE") and loc.i >= ent.start and loc.i < ent.end:
                    ubicacion_evento = ent.text

# ---- Salida de relaciones (estilo tabla)
relaciones = []

if sujeto_org and contraparte:
    relaciones.append((sujeto_org, "acuerdo con", contraparte))
if ubicacion_evento:
    relaciones.append(("Acuerdo", "ubicación", ubicacion_evento))
if objetivo:
    verbo_obj, objeto_obj = objetivo
    if objeto_obj:
        relaciones.append(("Acuerdo", "objetivo", f"{verbo_obj} {objeto_obj}"))
    if ubicacion_objetivo:
        relaciones.append(("Objetivo", "ubicación", ubicacion_objetivo))

print("\nRELACIONES:")
for r in relaciones:
    print(f" - {r[0]} → {r[1]} → {r[2]}")

ENTIDADES (texto, etiqueta):
 - ('Ecopetrol', 'ORG')
 - ('Bogotá', 'LOC')
 - ('Repsol', 'ORG')
 - ('Meta', 'LOC')

ENTIDADES (normalizadas):
 - Ecopetrol → Organización
 - Bogotá → Ubicación
 - Repsol → Organización
 - Meta → Ubicación

RELACIONES:
 - Acuerdo → ubicación → Meta


**Nota:** esta EI por reglas es deliberadamente simple para fines didácticos. En producción, conviene entrenar un modelo de extracción de relaciones (supervisado o instruction-tuned) y/o usar pattern matchers más robustos (por ejemplo, spacy.matcher/DependencyMatcher) y normalización de entidades.